In [17]:
import numpy as np
import pandas as pd
import math
from scipy.stats import poisson
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import plotly.express as px


In [18]:
def sqrt_func(i, j):
    time.sleep(1)
    return i + j


Parallel(n_jobs=2)(delayed(sqrt_func)(i, 5)
                   for i in range(5))


[5, 6, 7, 8, 9]

## Functions

In [19]:
I = 48  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

print(N)

101


In [20]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [21]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [22]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probS = calcExponentialLimit(beta*N)+1
print(probS)
calculate_p(beta, probS, precision)


1030


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [23]:
def calculate_v(p, beta, precision, precision_limit, N, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*N) + 1
  v = np.zeros((N+1, limit+d))
  add_v = np.zeros((N+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, N+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(N+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probS, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [8.03333902e-08, 4.08412610e-08, 1.97009237e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 1029)

In [24]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########

v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)


In [25]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, N, no_show, beta):
  return (I * d) + tardiness - (N * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, N):
  w = np.zeros((I+1, N+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= N
  return waitingtime

##########
## TEST ##
##########

calcWaitingtime(p_min, x, p, limit, I, N)


326.24609516473095

In [26]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess


##########
## TEST ##
##########

# calcFracExcess(p_min, I)


<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [27]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nN: {sum(x)},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results['objVal']

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probS, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [1 2 2 3 2 4 3 1 3 2 3 3 2 1 0 3 1 0 1 2 1 0 3 2 4 1 3 0 4 1 4 0 3 1 4 2 1
 0 3 3 4 2 4 4 0 3 2 3],
N: 101,
Objective value: 395.38953335375106,
Prob calculation time: 19.976305 sec,
Waiting time (timer): 326.24609516473095 (44.643055 sec),
Idle time (timer): -4.514841186902117 (0.000003 sec),
Tardiness (timer): 664.4851588130979 (0.000422 sec)



395.38953335375106

In [28]:
def createDist(arr, reverse=False):
    minX, maxX = arr.min(), arr.max()
    minY, maxY = 0, 1
    if(reverse):
        minY, maxY = maxY, minY
    # Scale to values between 0, 1 and make total sum equal to 1
    arrScaled = np.interp(arr, (minX, maxX), (minY, maxY))
    arrNorm = [x / arrScaled.sum() for x in arrScaled]
    return(arrNorm)
    

##########
## TEST ##
##########

arr = np.random.choice(10, 5)
dist1 = createDist(arr)
dist2 = createDist(arr, reverse=True)
arr, dist1, sum(dist1), dist2, sum(dist2)


(array([6, 1, 8, 7, 3]),
 [0.25, 0.0, 0.35000000000000003, 0.30000000000000004, 0.1],
 1.0000000000000002,
 [0.13333333333333336,
  0.46666666666666656,
  0.0,
  0.06666666666666668,
  0.33333333333333326],
 0.9999999999999998)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [29]:
# NEW
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    p2 = createDist(child2, reverse=True)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        j = np.random.choice(I, 1, p=p2)
        child1[i] += -1
        child2[j] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    p1 = createDist(child1, reverse=True)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        j = np.random.choice(I, 1, p=p1)
        child2[i] += -1
        child1[j] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[4 3 1 1 4 4 1 2 2 2] [1 0 4 4 2 2 2 3 2 3]


(array([3, 3, 3, 2, 3, 3, 1, 2, 2, 2]), array([2, 2, 4, 3, 2, 2, 2, 2, 2, 2]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [30]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[3 3 3 2 3 3 1 2 2 2]
[3 3 3 2 3 3 1 2 2 2]


## Classes

#### Suggesties: [get / set methods](https://www.geeksforgeeks.org/getter-and-setter-in-python/) voor parameters / met tests / nuttig voor optimaliseren hyper parameters, bv mbv bayesian stats

Create a data object to store and generate initial data.

In [31]:
class Data:
    def __init__(self, I=10, N=12, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5, nSwaps=4, mutationRate=0.1):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
        self.nSwaps, self.mutationRate = nSwaps, mutationRate
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W, self.nSwaps, self.mutationRate),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W', 'nSwaps', 'mutationRate'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8', 'i4', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]


array((101, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 60})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [33]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1

    def create_bw_schedule(self):
        self.x = np.ones(self.I).astype(int)
        self.x[0] = 2
        checkSum = np.sum(self.x)
        print(f'checkSum = {checkSum}')
        diff = checkSum - self.N
        print(f'diff = {diff}')
        if diff > 0:
            i = np.random.choice(range(1, self.I), diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] - 1
        if diff < 0:
            i = np.random.choice(range(1, self.I), -diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)


##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
[5 2 1 1 0 2 3 0 3 2 1 1 2 1 1 1 2 3 3 2 4 0 3 2 3 3 0 5 1 3 1 1 1 1 2 2 3
 4 4 1 4 5 0 3 3 4 0 2]
101
checkSum = 49
diff = -52


ValueError: Cannot take a larger sample than population when 'replace=False'

Create population of individuals

In [34]:
#NEW
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array(Parallel(n_jobs=7)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in self.individuals))
            
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array(Parallel(n_jobs=7)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v,
                                                                              N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in newIndividuals))
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


## Tests

In [35]:
##########
## TEST ##
##########

bestRank = []
dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [1 0 2 1 1 1 1 3 1 1],
N: 12,
Objective value: 31.310210744312858,
Prob calculation time: 0.133058 sec,
Waiting time (timer): 26.106522567810217 (0.068724 sec),
Idle time (timer): 1.7138989208154953 (0.000003 sec),
Tardiness (timer): 59.7138989208155 (0.000115 sec)

Schedule: [0 2 1 1 1 1 1 3 1 1],
N: 12,
Objective value: 34.608927991965714,
Prob calculation time: 0.129064 sec,
Waiting time (timer): 29.479815029236367 (0.068339 sec),
Idle time (timer): 4.9380409546950546 (0.000002 sec),
Tardiness (timer): 62.93804095469506 (0.000060 sec)

Schedule: [0 1 1 2 1 0 1 3 1 2],
N: 12,
Objective value: 33.452085442655424,
Prob calculation time: 0.125242 sec,
Waiting time (timer): 27.068771060974857 (0.094579 sec),
Idle time (timer): 5.035399824335997 (0.000002 sec),
Tardiness (timer): 63.035399824336 (0.000085 sec)

Schedule: [2 0 3 0 0 1 1 2 2 1],
N: 12,
Objective value: 30.469956213846103,
Prob calculation time: 0.132869 sec,
Waiting time (timer): 26.170723117622497 (0.117087 sec),

In [36]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([2, 0, 3, 0, 0, 1, 1, 2, 2, 1]), array([3, 0, 0, 0, 4, 0, 0, 2, 1, 2]), array([1, 0, 2, 1, 1, 1, 1, 3, 1, 1]), array([0, 1, 1, 2, 1, 0, 1, 3, 1, 2]), array([0, 2, 1, 1, 1, 1, 1, 3, 1, 1])] 
 [array([2, 0, 3, 0, 0, 1, 1, 2, 2, 1]), array([3, 0, 0, 0, 4, 0, 0, 2, 1, 2]), array([1, 0, 2, 1, 1, 1, 1, 3, 1, 1]), array([0, 1, 1, 2, 1, 0, 1, 3, 1, 2]), array([0, 2, 1, 1, 1, 1, 1, 3, 1, 1])]


In [37]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [1 0 2 1 1 1 1 3 1 1],
N: 12,
Objective value: 31.310210744312858,
Prob calculation time: 0.107024 sec,
Waiting time (timer): 26.106522567810217 (0.067333 sec),
Idle time (timer): 1.7138989208154953 (0.000002 sec),
Tardiness (timer): 59.7138989208155 (0.000062 sec)

Schedule: [0 2 1 1 1 1 1 3 1 1],
N: 12,
Objective value: 34.608927991965714,
Prob calculation time: 0.104370 sec,
Waiting time (timer): 29.479815029236367 (0.065875 sec),
Idle time (timer): 4.9380409546950546 (0.000002 sec),
Tardiness (timer): 62.93804095469506 (0.000064 sec)

Schedule: [0 1 1 2 1 0 1 3 1 2],
N: 12,
Objective value: 33.452085442655424,
Prob calculation time: 0.106502 sec,
Waiting time (timer): 27.068771060974857 (0.088989 sec),
Idle time (timer): 5.035399824335997 (0.000002 sec),
Tardiness (timer): 63.035399824336 (0.000062 sec)

Schedule: [2 0 3 0 0 1 1 2 2 1],
N: 12,
Objective value: 30.469956213846103,
Prob calculation time: 0.114103 sec,
Waiting time (timer): 26.170723117622497 (0.115249 sec),

[array([1, 1, 0, 1, 1, 1, 1, 2, 1, 3]),
 array([2, 0, 1, 2, 1, 0, 0, 2, 2, 2]),
 array([2, 2, 0, 0, 2, 0, 0, 2, 1, 3]),
 array([2, 0, 0, 0, 2, 1, 0, 3, 2, 2]),
 array([1, 0, 1, 0, 4, 0, 0, 1, 2, 3])]

In [38]:
# Initial population
bestRank = []
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    print(f'Generation: {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals


Schedule: [1 0 1 1 2 2 1 2 1 1],
N: 12,
Objective value: 31.376466119516294,
Prob calculation time: 0.099156 sec,
Waiting time (timer): 26.16618108157992 (0.072099 sec),
Idle time (timer): 1.7867511574526702 (0.000007 sec),
Tardiness (timer): 59.78675115745266 (0.000056 sec)

Schedule: [3 1 0 3 0 1 1 1 1 1],
N: 12,
Objective value: 32.73579144001231,
Prob calculation time: 0.102982 sec,
Waiting time (timer): 30.729706093659363 (0.072400 sec),
Idle time (timer): -0.05812321363474382 (0.000002 sec),
Tardiness (timer): 57.941876786365256 (0.000058 sec)

Schedule: [2 3 1 1 2 0 1 0 1 1],
N: 12,
Objective value: 33.98525498637119,
Prob calculation time: 0.100932 sec,
Waiting time (timer): 33.22856916576974 (0.084790 sec),
Idle time (timer): -0.05805919302736129 (0.000001 sec),
Tardiness (timer): 57.941940806972646 (0.000056 sec)

Schedule: [1 2 0 1 0 3 2 2 1 0],
N: 12,
Objective value: 30.98009058683204,
Prob calculation time: 0.098090 sec,
Waiting time (timer): 27.08839074667233 (0.106859 s

In [39]:

bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF


,Score,Schedule
0,29.847904,"[1, 2, 0, 0, 0, 2, 2, 2, 1, 2]"
1,29.204902,"[1, 2, 0, 0, 1, 1, 2, 2, 1, 2]"
2,28.690572,"[2, 0, 0, 1, 1, 2, 0, 1, 2, 3]"


In [40]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 1 1 0 1 0 3 1 0 2 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 3 0 1 0 1 0 0 1 0 0 0
 1 1 0 0 0 1 1 1 0 0 1],
N: 24,
Objective value: 13.670660186662104,
Prob calculation time: 1.734857 sec,
Waiting time (timer): 10.12409381801455 (0.352399 sec),
Idle time (timer): 32.03503776576048 (0.000002 sec),
Tardiness (timer): 8.03503776576047 (0.000111 sec)

Schedule: [0 0 0 1 0 0 1 1 0 1 1 0 0 1 0 2 0 1 2 1 0 0 0 0 0 0 0 1 0 0 1 0 1 2 0 0 1
 0 0 0 2 0 2 0 1 0 1 0],
N: 24,
Objective value: 17.795678695014647,
Prob calculation time: 1.724518 sec,
Waiting time (timer): 9.107063539349516 (0.433630 sec),
Idle time (timer): 39.588088798791404 (0.000002 sec),
Tardiness (timer): 15.588088798791397 (0.000111 sec)

Schedule: [0 0 0 0 0 0 0 0 2 1 1 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 0 0 2 0
 1 1 2 0 0 3 1 0 0 0 0],
N: 24,
Objective value: 31.738151132201573,
Prob calculation time: 1.703019 sec,
Waiting time (timer): 17.932571701527337 (0.393671 sec),
Idle time (timer): 56.941870752651084 (0.0000

In [41]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='LimeGreen')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [42]:
scheduleObject = Schedule(dataObject)
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
x = scheduleObject.get_schedule()
calcFitness(x, beta, precision, limit, v, N, no_show,
            I, d, eind, alpha_W, alpha_I, alpha_T)


checkSum = 49
diff = 25
i = [33 43 40 45 14 42  1 19 27 39 26 37  8 35 10 16 34 29  6 18 32 15 31 17
 24]
[2 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 1
 0 1 0 0 1 0 0 1 0 1 1]
24
Schedule: [2 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 1
 0 1 0 0 1 0 0 1 0 1 1],
N: 24,
Objective value: 14.20124282252053,
Prob calculation time: 21.712487 sec,
Waiting time (timer): 10.139566058147503 (1.052324 sec),
Idle time (timer): 32.90902733210254 (0.000003 sec),
Tardiness (timer): 8.909027332102546 (0.000439 sec)



14.20124282252053

In [43]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 10

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 0 0 2 0 1 1 0 0 0
 1 2 1 0 1 1 0 0 1 0 0],
N: 24,
Objective value: 21.673138018163726,
Prob calculation time: 1.730885 sec,
Waiting time (timer): 12.52694244515208 (0.175596 sec),
Idle time (timer): 43.770601733504805 (0.000003 sec),
Tardiness (timer): 19.77060173350483 (0.000127 sec)

Schedule: [0 1 0 0 1 0 1 3 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0
 1 3 0 1 1 0 0 0 0 0 0],
N: 24,
Objective value: 13.660671042569465,
Prob calculation time: 1.681483 sec,
Waiting time (timer): 16.549469067920423 (0.254493 sec),
Idle time (timer): 27.7348056923355 (0.000002 sec),
Tardiness (timer): 3.734805692335489 (0.000110 sec)

Schedule: [0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 4 1 1 1 1 0 0 0 2
 0 0 0 1 1 0 1 0 1 0 2],
N: 24,
Objective value: 39.663985179120345,
Prob calculation time: 1.665160 sec,
Waiting time (timer): 20.933098639084694 (0.333604 sec),
Idle time (timer): 68.15124287247744 (0.00000

In [44]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='RoyalBlue')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [45]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 20

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [2 0 1 1 1 1 1 1 0 2 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 2 1 1
 1 0 0 0 1 0 0 0 1 0 0],
N: 24,
Objective value: 13.37098658000623,
Prob calculation time: 1.597705 sec,
Waiting time (timer): 17.581603636046577 (0.232681 sec),
Idle time (timer): 26.563908542646004 (0.000002 sec),
Tardiness (timer): 2.5639085426459967 (0.000109 sec)

Schedule: [0 1 0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 2 0 1 0 0 1 0 2 0 2 0 1 1 0 0 0 1
 0 0 0 0 0 0 1 1 1 0 2],
N: 24,
Objective value: 20.566081638826574,
Prob calculation time: 1.592382 sec,
Waiting time (timer): 8.564245468740921 (0.308439 sec),
Idle time (timer): 44.56730575221701 (0.000002 sec),
Tardiness (timer): 20.567305752217013 (0.000121 sec)

Schedule: [0 0 0 0 3 1 0 1 1 0 0 1 1 2 1 0 0 1 0 0 0 1 0 0 2 2 0 1 0 0 0 0 1 0 0 0 1
 1 0 1 0 1 1 0 0 0 0 0],
N: 24,
Objective value: 16.382951550173413,
Prob calculation time: 1.574117 sec,
Waiting time (timer): 22.833018330228327 (0.345560 sec),
Idle time (timer): 28.082907030136795 (0.00

In [46]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='Tomato')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [47]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 1 0 1 0 2 0 0 1 0 1 1 1 2 1 1 0 0 0 1 1 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0
 1 0 1 0 0 0 0 1 1 0 1],
N: 24,
Objective value: 14.670459294205632,
Prob calculation time: 2.081203 sec,
Waiting time (timer): 12.926274315906133 (0.162939 sec),
Idle time (timer): 31.833249279738624 (0.000003 sec),
Tardiness (timer): 7.8332492797386335 (0.000128 sec)

Schedule: [2 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 2 0 1 0 2
 1 0 0 0 0 0 0 1 0 1 1],
N: 24,
Objective value: 14.994988441655902,
Prob calculation time: 2.041679 sec,
Waiting time (timer): 9.632519933615688 (0.239831 sec),
Idle time (timer): 34.56996744701604 (0.000002 sec),
Tardiness (timer): 10.569967447016047 (0.000110 sec)

Schedule: [1 1 0 1 1 0 0 0 0 2 0 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 2 0 1 0 0 0 0 1 0 0
 0 1 0 2 1 0 0 0 1 2 0],
N: 24,
Objective value: 16.897447691603464,
Prob calculation time: 2.055204 sec,
Waiting time (timer): 6.309982157774214 (0.315250 sec),
Idle time (timer): 39.95575804748964 (0.000

In [48]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkOrange')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [49]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 0 0 3 0 1 1 0 0 0 0 0 1 1 0 0 0 1 1 2 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 2
 0 1 2 0 0 0 0 1 0 0 1],
N: 24,
Objective value: 12.248142968548347,
Prob calculation time: 1.657253 sec,
Waiting time (timer): 8.480660921262386 (0.356665 sec),
Idle time (timer): 30.75979766673899 (0.000002 sec),
Tardiness (timer): 6.759797666738986 (0.000113 sec)

Schedule: [0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 2 2 0 0 0 2 1 1 1 0 1 0
 2 0 0 0 0 0 2 1 0 0 0],
N: 24,
Objective value: 23.523031955081414,
Prob calculation time: 1.659725 sec,
Waiting time (timer): 15.1557154494829 (0.400883 sec),
Idle time (timer): 45.10124295881377 (0.000004 sec),
Tardiness (timer): 21.101242958813753 (0.000111 sec)

Schedule: [1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 2 0 0 1 0 1 1 0 2 0 0 1 0 0 1 1 1 2 0
 2 0 0 0 0 0 0 2 0 0 0],
N: 24,
Objective value: 15.19432320057931,
Prob calculation time: 1.690970 sec,
Waiting time (timer): 11.264297991293999 (0.399359 sec),
Idle time (timer): 33.81434000676953 (0.000002 

In [50]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkCyan')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}p.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")
